This notebook intends to scrape the online renting information in vancouver and keep it to the personal notion notebook. The notion notebook has added a housing hunting template to keep track of the renting status.

It only scrape craiglist site

Please note that the searching criteria is based on personal needs only. May need to modify before usage

# Prerequisite
This section will keep the library and function for the runtime section. Please load it before run the runtime section

In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install chardet
!pip install DateTime
!pip install urllib3

import requests as request
from bs4 import BeautifulSoup as soup
import chardet
from datetime import datetime, timedelta
from urllib.request import urlopen, Request
from datetime import datetime
import time
import random
import json

# config
notion_token = "{notion_token}"
notion_database_id = "{notion_database_id}"
notion_create_record_url = 'https://api.notion.com/v1/pages'
maxRows=5 #end the scraping when collected ads reach maxRows
google_api_key = "{google_api_key}"
notion_header = {
    "Authorization": f"Bearer {notion_token}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-02-22"
    }

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
################# Craiglist Related Method #################
def scraping_craiglist(already_stored_house={}):    
    i=0
    rows = {}
    yesterday = (datetime.today() - timedelta(days=1)).date() #only get yesterday ads
    while True:        
        print(f"page {i+1}")
        #page 2 => s=120, page 3 => s=240
        page = "" if i==0 else f"s={i*120}&"
        searchPageUrl = f"https://vancouver.craigslist.org/search/hhh?{page}min_price=1500&max_price=3100&min_bedrooms=2&min_bathrooms=2&availabilityMode=0&sale_date=all+dates"
        content = get_pageContent(searchPageUrl)
        page = soup(content,'html.parser')
        resultCards = page.select('#search-results li.result-row')        
        for result in resultCards:      
          url = result.find('a', href=True, class_="result-title")["href"]
          if url in rows: continue # skip if duplicated ads
          if url in already_stored_house: continue # skip if duplicated ads with notion
          attribute = {"url": url}
          attribute["adsDate"] = result.find(class_="result-date")["datetime"]
          attribute["price"]  = format_price_to_int(result.find(class_="result-price").text)
          attribute["area"]  = get_area_tag(result.find(class_="result-hood").text)
          img = result.select(".result-image.gallery")
          if img: 
            if img[0].get("data-ids") is not None:
              img_id = (img[0].get("data-ids").split(",")[0])[2:]
              attribute["image"] = f"https://images.craigslist.org/{img_id}_600x450.jpg"

          adsDate = datetime.strptime(attribute["adsDate"], "%Y-%m-%d %H:%M").date()

          if adsDate > yesterday: continue
          if adsDate < yesterday: return rows # completed    

          row = scrapingDetailPage(attribute["url"], attribute)
          if row["isDelete"] == "N": # skip those deleted
            rows[url] = row              
          if len(rows) >= maxRows: return rows # Avoid sending too many request

        i+=1

    return rows


def scrapingDetailPage(link, attribute={}):     
    content = get_pageContent(link)
    page = soup(content,'html.parser')
    attribute["url"] = link
    if page.find(id="has_been_removed"):
      attribute["isDelete"] = "Y"
    else:
      attribute["isDelete"] = "N"   
    attrgroup = page.find_all(class_="attrgroup")    
    for attr in attrgroup:
      bubbles = attr.findChildren("span" , recursive=False)
      for bubble in bubbles:
        if bubble.text.lower().find("br") != -1:
          attribute["room"] = bubble.text
        elif bubble.text.lower().find("ft2") != -1:
          attribute["Sqft"] = format_sqft_to_int(bubble.text)
        elif bubble.text.lower().find("available") != -1:
          attribute["available"] = format_available_to_date(bubble.text)
        elif bubble.text == "cats are OK - purrr":
          attribute["canCat"] = "Y"
        elif bubble.text in ["apartment","townhouse","house","condo"]:
          attribute["houseType"] = bubble.text
    address = page.find("div",class_="mapaddress")
    if address is not None:
      attribute["address"] = address.text
    price = page.find("span",class_="price")
    if price is not None:
      attribute["price"] = format_price_to_int(price.text)
    googlemap = page.find('a', href=True, text='google map')
    map = page.find(id='map')
    if map is not None:
      latitude = map["data-latitude"]
      longitude = map["data-longitude"]
      attribute["googlemap"] = f"https://www.google.com/maps/search/{latitude},{longitude}"
      address = get_address_by_googlemap(latitude,longitude)
      if address is not None:
        attribute["address"] = address
    

    time.sleep(random.randint(1, 10)) #Avoid too many request within short period of time
    return attribute

def format_price_to_int(price):
    if price is None: return price 
    return int(price.strip("$").replace(",",""))
def format_sqft_to_int(price):
    if price is None: return price 
    return int(price.replace("ft2",""))
def format_available_to_date(date):
    print(date)
    temp = date.replace("available","").strip()
    temp = try_parsing_date(temp)
    return temp.strftime("%Y-%m-%d")

def try_parsing_date(text):
    today = datetime.today()
    try:
      temp = datetime.strptime(text, '%b %d')
      return temp.replace(year=today.year)
    except ValueError:
      try:
        return datetime.strptime(text, '%b %d, %Y')
      except ValueError:
        return today

def get_area_detailpage(page):
    area = page.select('.postingtitletext small')
    area = "" if not area else get_area_tag(area[0].text)
    return area

def get_address_by_googlemap(latitude,longitude):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={latitude},{longitude}&key={google_api_key}"
    json = get_json(url)
    if not json["results"]: return None
    return json["results"][0]["formatted_address"]

def get_area_tag(area):
    # tends to show more specific area
    REGION_TAG = ["metrotown", "brentwood","new westminster","sfu","richmond", 
                  "burnaby", "coquitlam", "surrey","langley","downtown","west vancouver",
                  "north vancouver","east vancouver","south vancouver"]
    if not area: return ""
    area = area.lower()
    for region in REGION_TAG:
      if region in area:
        return region
    return area

################# Helper method - Request related #################

def decode(s):
    d = chardet.detect(s)
    (cs, conf) = (d['encoding'], d['confidence'])
    if conf > 0.60:
      try:
          return s.decode( cs, errors = 'replace' )
      except Exception:
          pass 
    # force and return only ascii subset
    return ''.join( [ chr(i) if i < 128 else ' ' for i in s ])

def get_pageContent(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
    req = Request(url=url, headers=headers) 
    with urlopen(req) as page:
        raw = page.read()
    return decode(raw)
def get_json(url):
    resp = request.get(url=url)
    return resp.json() # Check the JSON Response Content documentation below


################# Notion Related Method #################

def get_houses_from_notion():  
    has_more = True
    ads_url_from_notion = {}
    nextcursor = None
    while has_more:
      url = f"https://api.notion.com/v1/databases/{notion_database_id}/query"
      data = {} if nextcursor is None else {"start_cursor": nextcursor}

      res = request.request("POST", url, headers=notion_header, data=json.dumps(data))
      response = res.json()
      results = response.get("results")
      ads_url_from_notion.update(
          {result.get("properties").get("Link").get("url"):True 
            for result in results 
            if result.get("properties").get("Link") is not None})
      has_more = response.get("has_more")
      nextcursor = response.get("next_cursor")
    return ads_url_from_notion

def post_notion_db(row): 
    print(row) 

    newPageData = {
      "parent": { "database_id": notion_database_id },
      "properties": {
        "Address": {
          "title": [
            {
              "text": {
                  "content": "" if row.get("address") is None else row.get("address") 
              }
            }
          ]
        },
        "Link": {
          "url":  row.get("url")
        },
        "type":{
          "select": {
            "name": "" if row.get("houseType") is None else row.get("houseType")
          }
        },
        "Room":{
          "select": {
            "name": "" if row.get("room") is None else row.get("room")
          }
        },
        "Neighbourhood":{
          "select": {
            "name": "" if row.get("area") is None else row.get("area")
          }
        },        
        "Status":{
          "select": {
            "name": "Brainstorming"
          }
        },
      }
    }

    if "image" in row:
      newPageData["properties"]["Image"] = {"files":[{
            "name":"image",
            "external":{ "url": row.get("image") },            
      }]}

    if "googlemap" in row:
      newPageData["properties"]["GoogleMap"] = {
          "url": row.get("googlemap")
      }

    if "price" in row:
      newPageData["properties"]["Rent"] = {
          "number": row.get("price")
      }

    if "Sqft" in row:
      newPageData["properties"]["Sqft"] = {
          "number": row.get("Sqft")
      }
      
    if "available" in row:
      newPageData["properties"]["Date Available"] = {
        "date": {
              "start": row.get("available")
        }
      }   
    
    data = json.dumps(newPageData)
    res = request.request("POST", notion_create_record_url, headers=notion_header, data=data)

# Runtime
## Scrape Craiglist
The below code will scrape yesterday vancouver housing ads from craiglist, and stored the information to personal notion notebook.

In [4]:
already_stored_house = get_houses_from_notion()
rows = scraping_craiglist(already_stored_house)

for url in rows:  
  post_notion_db(rows[url])
  time.sleep(1)

page 1
{'url': 'https://vancouver.craigslist.org/van/apa/d/burnaby-renovated-br-suite-garden-house/7498396832.html', 'adsDate': '2022-06-18 21:46', 'price': 2550, 'area': 'metrotown', 'image': 'https://images.craigslist.org/00Y0Y_91erEGIhPsMz_0kE0dL_600x450.jpg', 'isDelete': 'N', 'room': '3BR / 2Ba', 'Sqft': 930, 'available': '2022-08-01', 'houseType': 'house', 'address': '6951 Nelson Ave, Burnaby, BC V5J 4B9, Canada', 'googlemap': 'https://www.google.com/maps/search/49.220614,-122.994153'}
{'url': 'https://vancouver.craigslist.org/pml/apa/d/port-coquitlam-bedroom-bathroom-condo/7498392996.html', 'adsDate': '2022-06-18 21:14', 'price': 2500, 'area': 'coquitlam', 'image': 'https://images.craigslist.org/01111_aGhi0zpFzCcz_09G06y_600x450.jpg', 'isDelete': 'N', 'room': '2BR / 2Ba', 'Sqft': 840, 'available': '2022-08-01', 'canCat': 'Y', 'houseType': 'condo', 'address': '1426 Parkway Blvd, Coquitlam, BC V3E 3L8, Canada', 'googlemap': 'https://www.google.com/maps/search/49.296847,-122.804485'

## (TODO) Change state for Deleted Post
This section will go into every housing post from notion and change their status to 'Reject' if the post is already delete